In [70]:
import pandas as pd
import requests
import selenium
from selenium import webdriver  # Selenium es una herramienta para automatizar la interacción con navegadores web.
from webdriver_manager.chrome import ChromeDriverManager  # ChromeDriverManager gestiona la instalación del controlador de Chrome.
from selenium.webdriver.common.keys import Keys  # Keys es útil para simular eventos de teclado en Selenium.
from selenium.webdriver.support.ui import Select  # Select se utiliza para interactuar con elementos <select> en páginas web.
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException # Excepciones comunes de selenium que nos podemos encontrar 
from bs4 import BeautifulSoup
from time import sleep 
import re
import sys 
import os
sys.path.append(os.path.abspath("../src"))   
from datetime import datetime


import funciones as f

Sacar lista supers

In [7]:
url_super = "https://super.facua.org/"
res_facua = requests.get(url_super)
sopa_mercados = BeautifulSoup(res_facua.content, "html.parser")
lista_mercados = sopa_mercados.findAll("a", {"class" :"nav-link"})
lista_supers= [mercado.getText().lower() for mercado in lista_mercados]
lista_supers.pop()
lista_supers


['mercadona', 'alcampo', 'carrefour', 'hipercor', 'dia', 'eroski']

Sacar lista productos

In [ ]:
url_mercados = f"https://super.facua.org/mercadona/"
res_producto = requests.get(url_mercados)
sopa_productos = BeautifulSoup(res_producto.content, "html.parser")
lista_prod = sopa_productos.findAll("p", {"class" :"fw-bolder"})
lista_productos= [mercado.getText().lower().replace(" ", "-") for mercado in lista_prod]
for i, producto in enumerate(lista_productos,start=1):
    id_prod= i
    nombre_producto= producto
    print(nombre_producto)
    print(id_prod)


aceite-de-girasol
1
aceite-de-oliva
2
leche
3


Sacar lista variantes


In [ ]:
url_variante = "https://super.facua.org/mercadona/aceite-de-girasol/"
res_variante= requests.get(url_variante)
sopa_variante = BeautifulSoup(res_variante.content, "html.parser")
lista_var = sopa_variante.findAll("p", {"class" :"fw-bolder"})
lista_variantes= [variante.getText().lower().replace(" ", "-").replace(",", "").replace("º","").replace(".","") for variante in lista_var]
for i, variante in enumerate(lista_variantes, start=1):
    id_var= i
    nombre_variante= variante
    print(id_var)
    print(nombre_variante)

1
aceite-de-girasol-refinado-02-hacendado-1-l
2
aceite-de-girasol-refinado-02-hacendado-5-l


In [48]:
for mercado in lista_supers:
    df_tablas= pd.DataFrame()
    driver = webdriver.Chrome()
    url_mercados = f"https://super.facua.org/{mercado}/"
    driver.get(url_mercados)
    sleep(3)
    #aceptar cookie
    driver.find_element("css selector", "#rcc-confirm-button").click()
    #hacer aquí la lista de productos
    res_producto = requests.get(url_mercados)
    sopa_productos = BeautifulSoup(res_producto.content, "html.parser")
    lista_prod = sopa_productos.findAll("p", {"class" :"fw-bolder"})
    lista_productos= [prod.getText().lower().replace(" ", "-") for prod in lista_prod]

    for i, producto in enumerate(lista_productos):
        driver.execute_script("window.scrollBy(0, 200);")
        sleep(2)
        #selecciona producto
        driver.find_element("css selector", f"body > section:nth-child(4) > div > div.row.gx-4.gx-lg-5.row-cols-2.row-cols-md-3.row-cols-xl-4.justify-content-center > div:nth-child({i+1}) > div > div.card-footer.p-4.pt-0.border-top-0.bg-transparent > div > a").click()
        sopa_variante = BeautifulSoup(driver.page_source, "html.parser")
        lista_var = sopa_variante.findAll("p", {"class" :"fw-bolder"})
        lista_variantes= [variante.getText().lower().replace(" ", "-").replace(",", "").replace("º","").replace(".","") for variante in lista_var]
        lista_variantes_bien=[]
        for elemento in lista_variantes:
            if elemento.startswith("aceite") or elemento.startswith("leche"):
                lista_variantes_bien.append(elemento)
        for j ,variante in enumerate(lista_variantes_bien):
            while True:
                #selecciona variante
                try:
                    sleep(2)
                    driver.find_element("css selector", f"body > section:nth-child(4) > div > div:nth-child(7) > div:nth-child({j+1}) > div > div.card-footer.p-4.pt-0.border-top-0.bg-transparent > div > a").click()
                    break
                except:
                    driver.execute_script("window.scrollBy(0, 200);")
            sopa_mercados = BeautifulSoup(driver.page_source, "html.parser")
            tabla_general= sopa_mercados.findAll("table")
            tablaindiv=tabla_general[-1]
            #por aquí crear las columnas necesarias usando lista de productos y variante (mercado se pone formateado tal cual)
            df_tablas=pd.concat([df_tablas, f.sacar_tabla(tablaindiv,mercado,producto,variante)])
            driver.back()
        driver.back()    
    driver.close()
    df_tablas.to_csv(f'../datos/datos{mercado}.csv', index=False)


In [4]:
df_tablas.head()

NameError: name 'df_tablas' is not defined

Carga de datos

In [6]:
alcampo=pd.read_csv("../datos/datosalcampo.csv")
mercadona=pd.read_csv("../datos/datosmercadona.csv")
carrefour=pd.read_csv("../datos/datoscarrefour.csv")
dia=pd.read_csv("../datos/datosdia.csv")
eroski=pd.read_csv("../datos/datoseroski.csv")
hipercor=pd.read_csv("../datos/datoshipercor.csv")

In [53]:
alcampo.head()

,Día,Precio (€),Variación,supermercado,producto,variante
0,02/07/2024,"57,13",=,alcampo,aceite-de-oliva,aceite-de-oliva-virgen-extra-procedente-aceitu...
1,03/07/2024,"57,13",=,alcampo,aceite-de-oliva,aceite-de-oliva-virgen-extra-procedente-aceitu...
2,04/07/2024,"57,13",=,alcampo,aceite-de-oliva,aceite-de-oliva-virgen-extra-procedente-aceitu...
3,05/07/2024,"57,13",=,alcampo,aceite-de-oliva,aceite-de-oliva-virgen-extra-procedente-aceitu...
4,06/07/2024,"57,13",=,alcampo,aceite-de-oliva,aceite-de-oliva-virgen-extra-procedente-aceitu...


In [55]:
mercadona.head()

,Día,Precio (€),Variación,supermercado,producto,variante
0,12/07/2024,"1,45",=,mercadona,aceite-de-girasol,aceite-de-girasol-refinado-02-hacendado-1-l
1,13/07/2024,"1,45",=,mercadona,aceite-de-girasol,aceite-de-girasol-refinado-02-hacendado-1-l
2,14/07/2024,"1,45",=,mercadona,aceite-de-girasol,aceite-de-girasol-refinado-02-hacendado-1-l
3,15/07/2024,"1,45",=,mercadona,aceite-de-girasol,aceite-de-girasol-refinado-02-hacendado-1-l
4,16/07/2024,"1,45",=,mercadona,aceite-de-girasol,aceite-de-girasol-refinado-02-hacendado-1-l


In [56]:
eroski.head()

,Día,Precio (€),Variación,supermercado,producto,variante
0,12/07/2024,"1,89",=,eroski,aceite-de-girasol,aceite-alto-oleico-eroski-botella-1-litro
1,13/07/2024,"1,89",=,eroski,aceite-de-girasol,aceite-alto-oleico-eroski-botella-1-litro
2,14/07/2024,"1,89",=,eroski,aceite-de-girasol,aceite-alto-oleico-eroski-botella-1-litro
3,15/07/2024,"1,89",=,eroski,aceite-de-girasol,aceite-alto-oleico-eroski-botella-1-litro
4,16/07/2024,"1,89",=,eroski,aceite-de-girasol,aceite-alto-oleico-eroski-botella-1-litro


In [57]:
dia.head()

,Día,Precio (€),Variación,supermercado,producto,variante
0,29/08/2024,"1,99",=,dia,aceite-de-girasol,aceite-de-girasol-alto-oleico-especial-freidor...
1,30/08/2024,"1,99",=,dia,aceite-de-girasol,aceite-de-girasol-alto-oleico-especial-freidor...
2,31/08/2024,"1,99",=,dia,aceite-de-girasol,aceite-de-girasol-alto-oleico-especial-freidor...
3,01/09/2024,"1,99",=,dia,aceite-de-girasol,aceite-de-girasol-alto-oleico-especial-freidor...
4,02/09/2024,"1,99",=,dia,aceite-de-girasol,aceite-de-girasol-alto-oleico-especial-freidor...


In [58]:
hipercor.head()

,Día,Precio (€),Variación,supermercado,producto,variante
0,02/07/2024,"14,94",=,hipercor,aceite-de-oliva,aceite-de-oliva-virgen-extra-picual-casa-junca...
1,11/07/2024,"14,94",=,hipercor,aceite-de-oliva,aceite-de-oliva-virgen-extra-picual-casa-junca...
2,12/07/2024,"14,94",=,hipercor,aceite-de-oliva,aceite-de-oliva-virgen-extra-picual-casa-junca...
3,13/07/2024,"14,94",=,hipercor,aceite-de-oliva,aceite-de-oliva-virgen-extra-picual-casa-junca...
4,14/07/2024,"14,94",=,hipercor,aceite-de-oliva,aceite-de-oliva-virgen-extra-picual-casa-junca...


In [59]:
carrefour.head()

,Día,Precio (€),Variación,supermercado,producto,variante
0,12/07/2024,"21,40",=,carrefour,aceite-de-girasol,aceite-de-girasol-capicua-garrafa-5-l
1,13/07/2024,"21,40",=,carrefour,aceite-de-girasol,aceite-de-girasol-capicua-garrafa-5-l
2,14/07/2024,"21,40",=,carrefour,aceite-de-girasol,aceite-de-girasol-capicua-garrafa-5-l
3,15/07/2024,"21,40",=,carrefour,aceite-de-girasol,aceite-de-girasol-capicua-garrafa-5-l
4,16/07/2024,"21,40",=,carrefour,aceite-de-girasol,aceite-de-girasol-capicua-garrafa-5-l


Creación Data Frame Total y Exploración inicial

In [7]:
df= pd.DataFrame()
df_total= pd.concat([df,carrefour,mercadona,eroski,dia,hipercor,alcampo])

In [8]:
df_total.head()

,Día,Precio (€),Variación,supermercado,producto,variante
0,12/07/2024,"21,40",=,carrefour,aceite-de-girasol,aceite-de-girasol-capicua-garrafa-5-l
1,13/07/2024,"21,40",=,carrefour,aceite-de-girasol,aceite-de-girasol-capicua-garrafa-5-l
2,14/07/2024,"21,40",=,carrefour,aceite-de-girasol,aceite-de-girasol-capicua-garrafa-5-l
3,15/07/2024,"21,40",=,carrefour,aceite-de-girasol,aceite-de-girasol-capicua-garrafa-5-l
4,16/07/2024,"21,40",=,carrefour,aceite-de-girasol,aceite-de-girasol-capicua-garrafa-5-l


**RESULTADOS INICIALES DE EXPLORACION**    


Contamos con un Data Frame de 59382 filas y 6 columnas.

*Nulos:*  

- No cuenta con ningun dato nulo que tengamos que tratar


*Duplicados:*  

- Contamos con 193 filas duplicadas lo que supone un 0,33% del total de filas.  
- Tras analizarlos y comprobar la web, nos dimos cuenta de que los podemos eliminar ya que en la propia web aparecen productos duplicados(Nos quedamos con un Data Frame con 59189 filas y 6 columnas). 

*Formatos:*  

- En la columna variacion quise convertir los iguales a 0, quedarme solo con los porcentajes y luego ya pasar a float.  
- En la columna precio cambié las , por los . y los pasé a float.   
- En la columna Día la fecha debería estar en formato fecha así que la cambié. 





Tratamos los duplicados

In [12]:
df_total.shape

(59382, 6)

In [16]:
round((df_total.duplicated().sum()/len(df_total))*100, 2)

np.float64(0.33)

In [27]:
df_total[df_total.duplicated()].head()

,Día,Precio (€),Variación,supermercado,producto,variante
11616,15/07/2024,"1,08",=,carrefour,leche,leche-desnatada-central-lechera-asturiana-brik...
11617,16/07/2024,"1,08",=,carrefour,leche,leche-desnatada-central-lechera-asturiana-brik...
11618,17/07/2024,"1,08",=,carrefour,leche,leche-desnatada-central-lechera-asturiana-brik...
11619,18/07/2024,"1,08",=,carrefour,leche,leche-desnatada-central-lechera-asturiana-brik...
11620,19/07/2024,"1,08",=,carrefour,leche,leche-desnatada-central-lechera-asturiana-brik...


In [29]:
df_total.drop_duplicates().shape

(59189, 6)

Formateamos

In [62]:
df_total.info()

<class 'pandas.core.frame.DataFrame'>
Index: 59382 entries, 0 to 214
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Día           59382 non-null  object
 1   Precio (€)    59382 non-null  object
 2   Variación     59382 non-null  object
 3   supermercado  59382 non-null  object
 4   producto      59382 non-null  object
 5   variante      59382 non-null  object
dtypes: object(6)
memory usage: 3.2+ MB


In [32]:
df_total["Precio (€)"]=df_total["Precio (€)"].str.replace(",", ".").astype(float)



In [37]:
df_total.sample(5)

,Día,Precio (€),Variación,supermercado,producto,variante
11724,07/08/2024,1.09,=,carrefour,leche,leche-desnatada-central-lechera-asturiana-sin-...
4376,06/08/2024,9.18,=,mercadona,leche,leche-semidesnatada-sin-lactosa-hacendado-9-l
6529,25/09/2024,1.15,"-0,15 (11,54%)",dia,leche,leche-desnatada-sin-lactosa-central-lechera-as...
293,03/10/2024,1.63,=,eroski,aceite-de-girasol,aceite-de-girasol-eroski-botella-1-litro
17778,05/09/2024,1.09,=,carrefour,leche,leche-semidesnatada-puleva-sin-gluten-sin-lact...


In [66]:
df_total["Variación"]=df_total["Variación"].str.replace("+","").str.replace(",", ".").str.split(" ").str[0].str.replace("=", "0").astype(float)

In [76]:
df_total["Día"]=pd.to_datetime(df_total["Día"], dayfirst= True)

In [77]:
df_total.info()

<class 'pandas.core.frame.DataFrame'>
Index: 59382 entries, 0 to 214
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Día           59382 non-null  datetime64[ns]
 1   Precio (€)    59382 non-null  float64       
 2   Variación     59382 non-null  float64       
 3   supermercado  59382 non-null  object        
 4   producto      59382 non-null  object        
 5   variante      59382 non-null  object        
dtypes: datetime64[ns](1), float64(2), object(3)
memory usage: 5.2+ MB


In [68]:
df_total.describe().T

,count,mean,std,min,25%,50%,75%,max
Precio (€),59382.0,7.931182,10.765216,0.69,1.38,3.8,9.75,64.59
Variación,59382.0,-0.004942,0.528967,-20.69,0.00,0.0,0.00,20.69


In [69]:
df_total.describe(include="O").T

,count,unique,top,freq
Día,59382,126,14/08/2024,604
supermercado,59382,6,eroski,23679
producto,59382,3,leche,31185
variante,59382,702,aceite-de-oliva-virgen-extra-picual-casa-junca...,207
